In [1]:
library("reticulate")
use_python("/net/mraid14/export/data/users/eladch/tools/CO7/python3/3.7.5/bin/python", required=TRUE)
library("anndata")
sc = import("scanpy")
mc2 = import("metacells")

In [2]:
library("metacell")
library("devtools")
library("Matrix")
library("data.table")
library("tidyverse")

Loading required package: usethis

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.3     v dplyr   1.0.9
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()   masks data.table::between()
x tidyr::expand()    masks Matrix::expand()
x dplyr::filter()    masks stats::filter()
x dplyr::first()     masks data.table::first()
x dplyr::lag()       masks stats::lag()
x dplyr::last()      masks data.table::last()
x tidyr::pack()      masks Matrix::pack()
x readr::read_csv()  masks anndata::read_csv()
x purrr::transpose() masks data.table::transpose()
x tidyr::unpack()    masks Matrix::unpack()



In [3]:
##### TODO change mat_id and scdb_dir to what you need #####
setwd("/net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/Submission/")
scdb_dir = "/net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/Submission/scrna_db/"
fig_dir <- "/net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/Submission/figs/"
if(!dir.exists(fig_dir)) {
    dir.create(fig_dir)
  }

scfigs_init(fig_dir)

mat_id = "mat.iETX3.h5ad"
mc_id = "mc.iETX3.h5ad"
######

In [4]:
MC2_metacell = anndata::read_h5ad(paste(scdb_dir,mc_id, sep = ""))
MC2_mat = anndata::read_h5ad(paste(scdb_dir,mat_id, sep = ""))

cells_mc = MC2_mat$obs$metacell
cells_mc = cells_mc + 1
cell_names = MC2_mat$obs_names
names(cells_mc) = cell_names
gene_names = MC2_metacell$var_names

In [5]:
outlier_cells = names(cells_mc)[cells_mc == 0]
cells_mc_filt = cells_mc[cells_mc > 0]

umis2 = as.matrix(MC2_metacell$X)
fractions = umis2 / rowSums(umis2)
fractions = t(fractions)

In [15]:
pre_mat = scm_new_matrix(mat = t(MC2_mat$X),MC2_mat$obs,stat_type = 'umis')

In [16]:
scdb_init(scdb_dir,force_reinit = T)
scdb_add_mat(id = "iETX3_MC1", mat = pre_mat)

initializing scdb to /net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/Submission/scrna_db/



In [17]:
scdb_ls("mat")

[1] "mat.SynEmb3"        "mat.SynEmb3_embexe" "mat.embexe"        
[4] "mat.iETX3.h5ad"     "mat.iETX3_MC1"     


In [18]:
scdb_init(scdb_dir,force_reinit = T)
MC1_mat = scdb_mat("iETX3_MC1")
MC1_mc_embexe <- scdb_mc("embexe_recolored")

scdb_add_mc("iETX3_MC1", tgMCCov(cells_mc_filt, c(outlier_cells, setdiff(MC1_mat@cells, c(outlier_cells, names(cells_mc_filt)))), MC1_mat))
MC1_metacell = scdb_mc("iETX3_MC1")
colnames(fractions) = colnames(MC1_metacell@e_gc)
MC1_metacell@e_gc = fractions
MC1_metacell@color_key <- MC1_mc_embexe@color_key
MC1_metacell@cell_names <- names(MC1_metacell@mc)
scdb_add_mc(id = "iETX3_MC1", mc = MC1_metacell)

initializing scdb to /net/mraid14/export/tgdata/users/hernan/wd_NEW/SynEmbs3/Submission/scrna_db/

add batch counts

compute footprints

compute absolute ps

compute coverage ps



In [19]:
bad_genes <- rownames(MC2_metacell$var)[MC2_metacell$var$forbidden_gene]
feature_genes <- rownames(MC2_metacell$var)[MC2_metacell$var$top_feature_gene]

write.table(bad_genes, file = "iETX3_bad_genes.txt", sep ="\t")
write.table(feature_genes, file = "iETX3_feature_genes.txt", sep ="\t")

# MC1_mat = scm_ignore_cells(scmat = MC1_mat, ig_cells = names(MC1_metacell@mc), reverse = T)
# MC1_mat = scm_ignore_genes(scmat = MC1_mat, ig_genes = bad_genes, reverse = F)
# scdb_add_mat(id = "iETX3", mat = MC1_mat)

T_vm = 0.1
T_top3 = 3
T_tot = 50
K = 100

mcell_add_gene_stat(mat_id = "iETX3", gstat_id = "iETX3", force = T)
mcell_gset_filter_varmean(gstat_id = "iETX3",gset_id = "iETX3_new", T_vm = T_vm, force_new = T)
mcell_gset_filter_cov(gstat_id = "iETX3", gset_id = "iETX3_new", T_tot = T_tot, T_top3 = T_top3)

gset_new = scdb_gset("iETX3_new")

mcell_add_cgraph_from_mat_bknn(mat_id = "iETX3",
                               gset_id = "iETX3_new", 
                               graph_id = "iETX3_new", 
                               K = K, 
                               dsamp = T)

Calculating gene statistics... 

will downsamp

done downsamp

will gen mat_n

done gen mat_n

done computing basic gstat, will compute trends

Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot):
"the standard deviation is zero"
Warning message in cor(x, ctot)

..done


will downsample the matrix, N= 246

will build balanced knn graph on 190881 cells and 577 genes, this can be a bit heavy for >20,000 cells



In [20]:
scdb_ls("mat","iETX3")
scdb_ls("mc","iETX3")
scdb_ls("gset","iETX3")
scdb_ls("gstat","iETX3")

[1] "mat.iETX3"      "mat.iETX3.h5ad" "mat.iETX3_MC1" 
[1] "mc.iETX3.h5ad" "mc.iETX3_MC1" 
[1] "gset.iETX3_new"
[1] "gstat.iETX3"


## Project on MC1 embexe atlas

In [21]:
wt_atlas = mcell_gen_atlas(mat_id = "embexe",
                           mc_id = "embexe_recolored",
                           gset_id  = "embexe",
                           mc2d_id= "embexe_recolored_umap")

In [ ]:
mcell_proj_on_atlas(mat_id = "iETX3_MC1", 
                    mc_id = "iETX3_MC1", 
                    atlas = wt_atlas, 
                    recolor_mc_id = "iETX3_MC1_recolored",
                    fig_cmp_dir = fig_dir,
                    max_entropy = 4, 
                    burn_cor = 0.6)

In [ ]:
tgconfig::override_params(config_file = "sing_emb.yaml",package = "metacell")

mcell_mc2d_force_knn(mc2d_id = "iETX3_MC1_recolored_umap",
                     mc_id = "iETX3_MC1_recolored",
                     graph_id = "iETX3_new")

mcell_mc2d_plot(mc2d_id = "iETX3_MC1_recolored_umap",
                plot_edges = T)